In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Load datasets
train = pd.read_csv("/content/drive/My Drive/Datasets/train.csv")
test = pd.read_csv("/content/drive/My Drive/Datasets/test.csv")

# Show first few rows
train.head()

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,0,42,technician,married,secondary,no,7,no,no,cellular,25,aug,117,3,-1,0,unknown,0
1,1,38,blue-collar,married,secondary,no,514,no,no,unknown,18,jun,185,1,-1,0,unknown,0
2,2,36,blue-collar,married,secondary,no,602,yes,no,unknown,14,may,111,2,-1,0,unknown,0
3,3,27,student,single,secondary,no,34,yes,no,unknown,28,may,10,2,-1,0,unknown,0
4,4,26,technician,married,secondary,no,889,yes,no,cellular,3,feb,902,1,-1,0,unknown,1


In [ ]:
test.head()

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,750000,32,blue-collar,married,secondary,no,1397,yes,no,unknown,21,may,224,1,-1,0,unknown
1,750001,44,management,married,tertiary,no,23,yes,no,cellular,3,apr,586,2,-1,0,unknown
2,750002,36,self-employed,married,primary,no,46,yes,yes,cellular,13,may,111,2,-1,0,unknown
3,750003,58,blue-collar,married,secondary,no,-1380,yes,yes,unknown,29,may,125,1,-1,0,unknown
4,750004,28,technician,single,secondary,no,1950,yes,no,cellular,22,jul,181,1,-1,0,unknown


In [ ]:
print(train.describe())

                  id            age        balance            day  \
count  750000.000000  750000.000000  750000.000000  750000.000000   
mean   374999.500000      40.926395    1204.067397      16.117209   
std    216506.495284      10.098829    2836.096759       8.250832   
min         0.000000      18.000000   -8019.000000       1.000000   
25%    187499.750000      33.000000       0.000000       9.000000   
50%    374999.500000      39.000000     634.000000      17.000000   
75%    562499.250000      48.000000    1390.000000      21.000000   
max    749999.000000      95.000000   99717.000000      31.000000   

            duration       campaign          pdays       previous  \
count  750000.000000  750000.000000  750000.000000  750000.000000   
mean      256.229144       2.577008      22.412733       0.298545   
std       272.555662       2.718514      77.319998       1.335926   
min         1.000000       1.000000      -1.000000       0.000000   
25%        91.000000       1.0000

In [ ]:
print(train.isnull().sum())

id           0
age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64


In [ ]:
print(train['age'].value_counts(normalize=True));

age
32    0.049647
33    0.046015
31    0.045735
34    0.045508
35    0.044076
        ...   
90    0.000008
92    0.000007
94    0.000005
95    0.000004
91    0.000001
Name: proportion, Length: 78, dtype: float64


In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

In [ ]:
ID_COL = "id"
TARGET = "y"

X = train.drop(columns=[TARGET])
y = train[TARGET].astype(int)

In [ ]:
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()
cat_idx = [X.columns.get_loc(c) for c in cat_cols]

print("Categorical columns:", cat_cols)


Categorical columns: ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']


In [ ]:
params = dict(
    loss_function="Logloss",  # Binary classification
    eval_metric="AUC",        # Optimize for ROC AUC
    depth=8,                  # Tree depth (controls complexity)
    learning_rate=0.05,       # Step size
    l2_leaf_reg=3.0,          # Regularization
    iterations=20000,         # Max boosting rounds
    random_seed=42,           # Reproducibility
    od_type="Iter",           # Early stopping
    od_wait=1000,             # Stop if no improvement for 1000 rounds
    verbose=500               # Log progress every 500 rounds
)


In [ ]:
from catboost import CatBoostClassifier, Pool

# --- Light & Fast Parameters ---
params = {
    'iterations': 300,        # fewer boosting rounds
    'depth': 6,               # shallower trees
    'learning_rate': 0.1,     # faster convergence
    'loss_function': 'Logloss',
    'eval_metric': 'AUC',
    'random_seed': 42,
    'task_type': 'CPU',       # change to 'GPU' if available
    'thread_count': -1,       # use all cores
    'verbose': 100
}

# --- Train single model ---
train_pool = Pool(X, y, cat_features=cat_idx)
test_pool  = Pool(test, cat_features=cat_idx)

model = CatBoostClassifier(**params)
model.fit(train_pool)

# --- Predictions ---
oof = model.predict_proba(train_pool)[:, 1]
preds = model.predict_proba(test_pool)[:, 1]


0:	total: 2.31s	remaining: 11m 32s
100:	total: 2m	remaining: 3m 57s
200:	total: 3m 56s	remaining: 1m 56s
299:	total: 5m 51s	remaining: 0us


In [ ]:
import pandas as pd

# Create submission DataFrame
submission = pd.DataFrame({
    'id': test['id'],      # 'id' column from test dataset
    'target': preds        # your prediction probabilities from CatBoost
})

# Save to CSV
submission.to_csv('submission.csv', index=False)
print("Submission file saved as submission.csv")


Submission file saved as submission.csv


In [ ]:
submission = pd.DataFrame({
    'id': test['id'],
    'target': (preds > 0.5).astype(int)  # threshold at 0.5
})
submission.to_csv('submission.csv', index=False)


In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score

# Validation predictions
val_preds = model.predict(X_val)  # Class labels (0 or 1)
val_probs = model.predict_proba(X_val)[:, 1]  # Probabilities

# Accuracy
acc = accuracy_score(y_val, val_preds)
print("Validation Accuracy:", acc)

# AUC score (useful for binary problems)
auc = roc_auc_score(y_val, val_probs)
print("Validation AUC:", auc)


Validation Accuracy: 0.934224
Validation AUC: 0.9650561018355848


NameError: name 'show' is not defined

In [ ]:
import os
os.environ['kaggle (1).json'] = "/content"


In [ ]:
!kaggle competitions submit -c binary-classification-with-a-bank-dataset -f submission.csv -m "CatBoost submission from Colab"


Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 4, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.12/dist-packages/kaggle/__init__.py", line 6, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 434, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.config/kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


In [ ]:
import time
timestamp = time.strftime("%Y%m%d_%H%M%S")
new_name = f"submission_{timestamp}.csv"
os.rename("submission.csv", new_name)
print(f"File renamed to {new_name}")

# Submit the renamed file
!kaggle competitions submit -c binary-classification-with-a-bank-dataset -f {new_name} -m "Submission {timestamp}"


File renamed to submission_20250830_084607.csv
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 4, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.12/dist-packages/kaggle/__init__.py", line 6, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 434, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.config/kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


In [ ]:
from google.colab import files
files.upload()


Saving kaggle (1).json to kaggle (1).json


{'kaggle (1).json': b'{"username":"ghulamqadirjunejo","key":"322cbbfbb0f98a1c87de4cebda00b970"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle (1).json


mv: cannot stat 'kaggle.json': No such file or directory
/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `chmod 600 ~/.kaggle/kaggle(1).json'


In [ ]:
!head submission_20250830_084607.csv

id,target
750000,0
750001,0
750002,0
750003,0
750004,0
750005,0
750006,0
750007,0
750008,0


In [ ]:
files.download('submission_20250830_084607.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>